# Cleaning Tweets - NLP

### Importing Libraries

In [1]:
import numpy as np
import random
import _pickle as pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
import nltk
import string
import re
from gensim.models import Word2Vec
from sklearn.model_selection import GridSearchCV

### Importing DataFrames

In [2]:
# Loading in the first DF
with open("city1_df.pkl",'rb') as fp:
    city_1 = pickle.load(fp)

# Loading in the second DF
with open("city2_df.pkl",'rb') as fp:
    city_2 = pickle.load(fp)

### Checking the DataFrames

In [3]:
city_1

,cashtags,conversation_id,created_at,date,day,geo,hashtags,hour,id,link,...,search,source,timezone,tweet,user_id,user_id_str,user_rt,user_rt_id,username,City
0,[],1162465481598615554,1570108924000,2019-10-03 09:22:04,7,,[],19,1179748453049520130,https://twitter.com/PegasusLace/status/1179748...,...,Trump,,EDT,WW1 and what nationalism did to Europe 🙄 pic.t...,488132175,488132175,,,PegasusLace,Seattle
1,[],1179422987684077568,1570108874000,2019-10-03 09:21:14,7,,[],05,1179748242168242176,https://twitter.com/DaleLindholm/status/117974...,...,Trump,,EDT,You lost the popular vote 🙄,3070803430,3070803430,,,DaleLindholm,Seattle
2,[],1179748161042227201,1570108854000,2019-10-03 09:20:54,7,,[],00,1179748161042227201,https://twitter.com/seattlepi/status/117974816...,...,Trump,,EDT,"Trump touts support for Medicare, slams ‘Medic...",10648962,10648962,,,seattlepi,Seattle
3,[],1179748132285927424,1570108848000,2019-10-03 09:20:48,6,,[#trumpmeltown],22,1179748132285927424,https://twitter.com/cismontaine/status/1179748...,...,Trump,,EDT,"The Fins and the press be all, just a quick qu...",1007815687,1007815687,,,cismontaine,Seattle
4,[],1179748108416237568,1570108842000,2019-10-03 09:20:42,6,,"[#leningradlindsey, #moscowmitch]",21,1179748108416237568,https://twitter.com/ShelliDrummer4/status/1179...,...,Trump,,EDT,The party of collusion. @LindseyGrahamSC #Leni...,972820757662253056,972820757662253056,,,ShelliDrummer4,Seattle
5,[],1179748105006149632,1570108841000,2019-10-03 09:20:41,6,,"[#prisoniscoming, #byebyebye]",20,1179748105006149632,https://twitter.com/sunmaya11/status/117974810...,...,Trump,,EDT,#Prisoniscoming @realDonaldTrump #byebyebye ht...,120236353,120236353,,,sunmaya11,Seattle
6,[],1159870945630117888,1570108791000,2019-10-03 09:19:51,6,,[],06,1179747895634907138,https://twitter.com/devinher/status/1179747895...,...,Trump,,EDT,This is kind of genius. With Trump’s corruptio...,15923351,15923351,,,devinher,Seattle
7,[],1179742652159463424,1570108783000,2019-10-03 09:19:43,6,,[],04,1179747860629180416,https://twitter.com/jyeater/status/11797478606...,...,Trump,,EDT,"In a profile I have, for the political prefere...",22212186,22212186,,,jyeater,Seattle
8,[],1179721004291170304,1570108754000,2019-10-03 09:19:14,5,,[],20,1179747739522883584,https://twitter.com/katsrx7/status/11797477395...,...,Trump,,EDT,"Dude your from bumfuck somewhere, get out of o...",43601584,43601584,,,katsrx7,Seattle
9,[],1179744506440228865,1570108751000,2019-10-03 09:19:11,5,,[],19,1179747729032892418,https://twitter.com/AnnAnnChe/status/117974772...,...,Trump,,EDT,Every article coming out is predicting Biden i...,909673932042543105,909673932042543105,,,AnnAnnChe,Seattle


In [4]:
city_2

,cashtags,conversation_id,created_at,date,day,geo,hashtags,hour,id,link,...,search,source,timezone,tweet,user_id,user_id_str,user_rt,user_rt_id,username,City
0,[],1179734856789626880,1570109383000,2019-10-03 09:29:43,6,,[],03,1179750379036299265,https://twitter.com/jonmbeard/status/117975037...,...,Trump,,EDT,Different Ukraine President. Don't confuse the...,195969147,195969147,,,jonmbeard,Jacksonville
1,[],1179740389265281025,1570109073000,2019-10-03 09:24:33,2,,[],13,1179749079036911620,https://twitter.com/ophs1977/status/1179749079...,...,Trump,,EDT,Impeachment just starts the process.. it will ...,42423448,42423448,,,ophs1977,Jacksonville
2,[],1179748994077122563,1570109053000,2019-10-03 09:24:13,2,,[],07,1179748994077122563,https://twitter.com/JakeBrannan5499/status/117...,...,Trump,,EDT,"You can certainly think that, but everyone els...",773610004075208704,773610004075208704,,,JakeBrannan5499,Jacksonville
3,[],1179748955506262016,1570109044000,2019-10-03 09:24:04,2,,"[#trumpmeltdown, #rally, #dallas]",05,1179748955506262016,https://twitter.com/SeaWolfLestat/status/11797...,...,Trump,,EDT,#TrumpMeltdown Prediction: @realDonaldTrump wi...,64462185,64462185,,,SeaWolfLestat,Jacksonville
4,[],1179542037479202816,1570109016000,2019-10-03 09:23:36,1,,[],21,1179748840506892288,https://twitter.com/SandraLeeKoeni2/status/117...,...,Trump,,EDT,Absolutely,942660351702601728,942660351702601728,,,SandraLeeKoeni2,Jacksonville
5,[],1179502966606352386,1570108962000,2019-10-03 09:22:42,1,,[],06,1179748612865167366,https://twitter.com/Bridgett76mill/status/1179...,...,Trump,,EDT,Facebook still has the video. Twitter did this...,1150434716572631040,1150434716572631040,,,Bridgett76mill,Jacksonville
6,[],1179748151600828422,1570108852000,2019-10-03 09:20:52,6,,[],23,1179748151600828422,https://twitter.com/jonathantony/status/117974...,...,Trump,,EDT,This is sadly the only time many of these kids...,57688276,57688276,,,jonathantony,Jacksonville
7,[],1179502966606352386,1570108809000,2019-10-03 09:20:09,6,,[],11,1179747968582332419,https://twitter.com/SharonS11157/status/117974...,...,Trump,,EDT,Then you've been watching too much of the libe...,434269162,434269162,,,SharonS11157,Jacksonville
8,[],1179747957266096128,1570108806000,2019-10-03 09:20:06,6,,[],11,1179747957266096128,https://twitter.com/iatwah7/status/11797479572...,...,Trump,,EDT,Doesnt Trump deserve forgiveness? https://www...,552978692,552978692,,,iatwah7,Jacksonville
9,[],1179744080877805568,1570108757000,2019-10-03 09:19:17,5,,[],21,1179747753573978113,https://twitter.com/jim_jax/status/11797477535...,...,Trump,,EDT,That’s almost a no-brainer. Trump is likely g...,614694471,614694471,,,jim_jax,Jacksonville


## Cleaning and NLP

### Function Balancing the class values

In [5]:
def city_balance(city_1, city_2):
    """
    Balances the number of unique tweets from each city
    Removes brief tweets
    """
#     # Removing tweets with less than 10 characters
#     city_1 = city_1[city_1.tweet.map(len)>10]
#     city_2 = city_2[city_2.tweet.map(len)>10]
    
#     # Removing tweets from the same user
#     city_1 = city_1.drop_duplicates('user_id')
#     city_2 = city_2.drop_duplicates('user_id')

    # Checking for class balance
    dif = abs(len(city_1) - len(city_2))

    # Randomly dropping rows from the DF with a greater number of rows
    if len(city_1) > len(city_2):
        city_1 = city_1.drop(random.sample(list(city_1.index), dif))
    elif len(city_2) > len(city_1):
        city_2 = city_2.drop(random.sample(list(city_2.index), dif))
    else:
        print("DFs are balanced")
    
    print(f"DF Lengths:\nCity 1 = {len(city_1)}\nCity 2 = {len(city_2)}")

    return city_1, city_2

In [6]:
city_1, city_2 = city_balance(city_1, city_2)

DF Lengths:
City 1 = 10000
City 2 = 10000


In [7]:
# Combining both dataframes
main_df = city_1.append(city_2)
main_df[['tweet', 'City']]

,tweet,City
0,WW1 and what nationalism did to Europe 🙄 pic.t...,Seattle
1,You lost the popular vote 🙄,Seattle
2,"Trump touts support for Medicare, slams ‘Medic...",Seattle
3,"The Fins and the press be all, just a quick qu...",Seattle
4,The party of collusion. @LindseyGrahamSC #Leni...,Seattle
5,#Prisoniscoming @realDonaldTrump #byebyebye ht...,Seattle
6,This is kind of genius. With Trump’s corruptio...,Seattle
7,"In a profile I have, for the political prefere...",Seattle
8,"Dude your from bumfuck somewhere, get out of o...",Seattle
9,Every article coming out is predicting Biden i...,Seattle


## Vectorizing
- TF_IDF Vectorizer

In [8]:
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [9]:
# Stopwords list
stop = stopwords.words('english')
stop += list(string.punctuation)

def clean_words(df_col, stop=stop):
    """
    Cleans tweets from dataframe column
    """
    # Lowercasing the words
    data = [tweet.lower() for tweet in df_col]
    # Removing URLs
    data = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', "", i) 
            for i in data]

    # Removing special characters
    data = [nltk.regexp_tokenize(i, "([a-zA-Z]+(?:'[a-z]+)?)") for i in data]

    # Lemmatizing words and removing stop words
    n = []
    for i in data:
        lst = ""
        for x in i:
            x = lemmatizer.lemmatize(x)
            if x not in stop:
                lst += (x+" ")
        n.append(lst)
        
    return n
    
data = clean_words(main_df.tweet, stop)

In [10]:
city1_clean = clean_words(city_1.tweet, stop)
city2_clean = clean_words(city_2.tweet, stop)

### Exporting the combined DataFrame and the cleaned tweets

In [467]:
with open("main_df.pkl", "wb") as fp:
    pickle.dump(main_df, fp)
    
with open("clean_tweets.pkl", "wb") as fp:
    pickle.dump(data, fp)

In [11]:
with open("city1_clean.pkl", "wb") as fp:
    pickle.dump(city1_clean, fp)
with open("city2_clean.pkl", "wb") as fp:
    pickle.dump(city2_clean, fp)